# Método dos Elementos Finitos - Trabalho 4

Universidade Federal Fluminense

Disciplina ministrada pelo Prof. Marco Ferro

<marcoferro@id.uff.br>

Aluno Noé de Lima

<noe_lima@id.uff.br>

Este trabalho visa aplicar o MEF ao Estado Plano de Tensões (EPT).

Primeiro semestre de 2020

\vfill

A célula a seguir configura o Jupyter-Notebook para exibir as equações matemáticas no formato do ambiente $\LaTeX$ e importa as bibliotecas necessárias.

In [1]:
import numpy as np
import sympy as sp
import json
from sympy.vector import CoordSys3D, divergence
sp.init_printing(use_latex='mathjax',latex_mode='equation*')
!uname -a

Linux DESKTOP-CR7O8A2 4.19.104-microsoft-standard #1 SMP Wed Feb 19 06:37:35 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux


\cleardoublepage

\tableofcontents

\cleardoublepage

# Introdução

O Estado Plano de Tenção - EPT ou o Estado Plano de Deformação - EPD, caracterizam-se por serem estruturas planares (no plano $xy$), com carregaemntos, reações de apoio, restrições e deformações igualmente planares. Tais estruturas têm, na direção $z$, espessura fixa constante $t$. As condições que definem e diferenciam o EPT e o EPD estão relacionadas às restrições na direção $z$.

Temos, na dereção $z$, das faces do plano, as seguintes variáveis de Tensão $\times$ Deformação: $\sigma_{z}$ e $\epsilon_{z}$.

Portanto, segue que:

No EPT: $\sigma_{z}=0$;

No EPD: $\epsilon_{z}=0$.

O EPT tem, portanto, suas faces livres, sem tensões de reação e liberdade de deformação. Tal é a situação em chapas, onde uma estrutura plana recebe carregamentos no mesmo plano da face, e não recebe carregamentos transversais (como nas placas ou lages). Nessa configuração, a espessura $t$ da chapa pode variar, pois não há tensão de reação na direção $z$.

Já o EPD é caracterizado por uma restrição de deformação, já que há reação de tensão na direção $z$ que restringe a deformação. Nessa configuração, portanto, a espessura $t$ não varia. Este é o caso onde o plano compõe uma seção transversão de uma estrutura maior, como um muro de arrimo ou uma barragem de represa. Neste caso, a espessura é fixa e definida como $t=1$, sendo as tensões definidas em termos de comprimento de muro ou barragem.

As variáveis simbólicas definidas abaixo serão utilizadas na análise subsequente.

In [2]:
E,nu = sp.symbols('E,nu')
G = E/(2*(1+nu))

Sendo,

* $E$ o Módulo de Elasticidade Longitudinal;

* $\nu$ a constante de Poisson do material; elastica; e

* $G$ o Módulo de Elasticidade Transversal.

In [3]:

sigma_x,sigma_y,sigma_z = sp.symbols('sigma_x,sigma_y,sigma_z')
tau_xy,tau_yz,tau_zx = sp.symbols('tau_xy,tau_yz,tau_zx')
epsilon_x,epsilon_y,epsilon_z = sp.symbols('epsilon_x,epsilon_y,epsilon_z')
gamma_xy,gamma_yz,gamma_zx = sp.symbols('gamma_xy,gamma_yz,gamma_zx')

Onde,

* $\sigma_{x}$, $\sigma_{y}$ e $\sigma_{z}$ são, respectivamente, as tensões normais nas direções $x$, $y$ e $z$;

* $\tau_{xy}$, $\tau_{yz}$ e $\tau_{zx}$ são, respectivamente, as tensões cisalhantes nas direções $xy$, $yz$ e $zx$;

* $\varepsilon_{x}$, $\varepsilon_{y}$ e $\varepsilon_{z}$ são, respectivamente, as deformações lineares em $x$ e $y$ e $z$;

* $\gamma_{xy}$, $\gamma_{yz}$ e $\gamma_{zx}$ são, respectivamente, as deformações angulares nas direções $xy$, $yz$ e $zx$.

A partir das restrições do sistema bidimensional, temos o seguinte Vetor de Tensões $\vec{\sigma}$:

In [4]:
sigma = sp.Matrix([sigma_x,sigma_y,tau_xy])
display(sigma)

⎡ σₓ ⎤
⎢    ⎥
⎢σ_y ⎥
⎢    ⎥
⎣τ_xy⎦

Bem como o Vetor de Deformações $\vec{\varepsilon}$:

In [5]:
epsilon = sp.Matrix([epsilon_x,epsilon_y,gamma_xy])
display(epsilon)

⎡ εₓ ⎤
⎢    ⎥
⎢ε_y ⎥
⎢    ⎥
⎣γ_xy⎦

Temos também a matriz $\mathbf{D}$, em função de $E_{var}$ e $\nu_{var}$:

In [6]:
E_var,nu_var = sp.symbols('E_var,nu_var')
D_var = (E_var/(1-nu_var**2))*sp.Matrix([[1,nu_var,0],[nu_var,1,0],[0,0,(1-nu_var)/2]])
display(D_var)

⎡   Eᵥₐᵣ    Eᵥₐᵣ⋅νᵥₐᵣ                 ⎤
⎢─────────  ─────────         0       ⎥
⎢        2          2                 ⎥
⎢1 - νᵥₐᵣ   1 - νᵥₐᵣ                  ⎥
⎢                                     ⎥
⎢Eᵥₐᵣ⋅νᵥₐᵣ     Eᵥₐᵣ                   ⎥
⎢─────────  ─────────         0       ⎥
⎢        2          2                 ⎥
⎢1 - νᵥₐᵣ   1 - νᵥₐᵣ                  ⎥
⎢                                     ⎥
⎢                           ⎛1   νᵥₐᵣ⎞⎥
⎢                      Eᵥₐᵣ⋅⎜─ - ────⎟⎥
⎢                           ⎝2    2  ⎠⎥
⎢    0          0      ───────────────⎥
⎢                                 2   ⎥
⎣                         1 - νᵥₐᵣ    ⎦

A equação do sistema, então, fica:

In [7]:
eq = sp.Eq(sigma, D_var*epsilon)
display(eq)

         ⎡ Eᵥₐᵣ⋅εₓ    Eᵥₐᵣ⋅ε_y⋅νᵥₐᵣ⎤
         ⎢───────── + ─────────────⎥
         ⎢        2             2  ⎥
         ⎢1 - νᵥₐᵣ      1 - νᵥₐᵣ   ⎥
         ⎢                         ⎥
         ⎢Eᵥₐᵣ⋅εₓ⋅νᵥₐᵣ    Eᵥₐᵣ⋅ε_y ⎥
⎡ σₓ ⎤   ⎢──────────── + ───────── ⎥
⎢    ⎥   ⎢         2             2 ⎥
⎢σ_y ⎥ = ⎢ 1 - νᵥₐᵣ      1 - νᵥₐᵣ  ⎥
⎢    ⎥   ⎢                         ⎥
⎣τ_xy⎦   ⎢            ⎛1   νᵥₐᵣ⎞   ⎥
         ⎢  Eᵥₐᵣ⋅γ_xy⋅⎜─ - ────⎟   ⎥
         ⎢            ⎝2    2  ⎠   ⎥
         ⎢  ────────────────────   ⎥
         ⎢               2         ⎥
         ⎣       1 - νᵥₐᵣ          ⎦

# Estado Plano de Tensão - EPT

As condições de contorno do EPT, conforme definido na introdução, podem ser definidas matematicamente como:

\begin{equation*}
    \left\{\begin{matrix}
        \sigma_{z}  & = &   0 \\
        \tau_{yz}   & = &   0 \\
        \tau_{zx}   & = &   0
    \end{matrix}\right.
\end{equation*}

E,

\begin{equation*}
    \varepsilon_{z} = -\frac{\nu}{E}\left(\sigma_{x}+\sigma_{y}\right) \neq 0
\end{equation*}

Nessas condições, temos:

\begin{equation*}
    \left\{\begin{matrix}
        E_{var}     & = &   E \\
        \nu_{var}   & = &   \nu
    \end{matrix}\right.
\end{equation*}

Logo, $\mathbf{D}_{EPT}$ =

In [8]:
ept = eq.subs(E_var,E).subs(nu_var,nu)
display(ept)

         ⎡ E⋅εₓ    E⋅ε_y⋅ν⎤
         ⎢────── + ───────⎥
         ⎢     2         2⎥
         ⎢1 - ν     1 - ν ⎥
         ⎢                ⎥
         ⎢E⋅εₓ⋅ν   E⋅ε_y  ⎥
⎡ σₓ ⎤   ⎢────── + ────── ⎥
⎢    ⎥   ⎢     2        2 ⎥
⎢σ_y ⎥ = ⎢1 - ν    1 - ν  ⎥
⎢    ⎥   ⎢                ⎥
⎣τ_xy⎦   ⎢        ⎛1   ν⎞ ⎥
         ⎢ E⋅γ_xy⋅⎜─ - ─⎟ ⎥
         ⎢        ⎝2   2⎠ ⎥
         ⎢ ────────────── ⎥
         ⎢          2     ⎥
         ⎣     1 - ν      ⎦

# Estado Plano de Deformação

As condições de contorno do EPD, conforme definido na introdução, podem ser definidas matematicamente como:

\begin{equation*}
    \left\{\begin{matrix}
        \varepsilon_{z}    & = &   0 \\
        \gamma_{yz}     & = &   0 \\
        \gamma_{zx}     & = &   0
    \end{matrix}\right.
\end{equation*}

E,

\begin{equation*}
    \sigma_{z} = \nu\left(\sigma_{x}+\sigma_{y}\right) \neq 0
\end{equation*}

Nessas condições, temos:

\begin{equation*}
    \left\{\begin{matrix}
        E_{var}     & = &   \frac{E}{1-\nu^{2}} \\
        \nu_{var}   & = &   \frac{\nu}{1-\nu}
    \end{matrix}\right.
\end{equation*}

Logo, $\mathbf{D}_{EPD}$ =

In [9]:
epd = eq.subs(E_var,E/(1-nu**2)).subs(nu_var,nu/(1-nu))
display(epd)

         ⎡           E⋅εₓ                          E⋅ε_y⋅ν             ⎤
         ⎢───────────────────────── + ─────────────────────────────────⎥
         ⎢         ⎛      2       ⎞                    ⎛      2       ⎞⎥
         ⎢⎛     2⎞ ⎜     ν        ⎟           ⎛     2⎞ ⎜     ν        ⎟⎥
         ⎢⎝1 - ν ⎠⋅⎜- ──────── + 1⎟   (1 - ν)⋅⎝1 - ν ⎠⋅⎜- ──────── + 1⎟⎥
         ⎢         ⎜         2    ⎟                    ⎜         2    ⎟⎥
         ⎢         ⎝  (1 - ν)     ⎠                    ⎝  (1 - ν)     ⎠⎥
         ⎢                                                             ⎥
         ⎢              E⋅εₓ⋅ν                          E⋅ε_y          ⎥
         ⎢───────────────────────────────── + ─────────────────────────⎥
⎡ σₓ ⎤   ⎢                 ⎛      2       ⎞            ⎛      2       ⎞⎥
⎢    ⎥   ⎢        ⎛     2⎞ ⎜     ν        ⎟   ⎛     2⎞ ⎜     ν        ⎟⎥
⎢σ_y ⎥ = ⎢(1 - ν)⋅⎝1 - ν ⎠⋅⎜- ──────── + 1⎟   ⎝1 - ν ⎠⋅⎜- ──────── + 1⎟⎥
⎢    ⎥   ⎢                 ⎜         2    ⎟        